<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div>


## Week 14 | Lab : Convolutional Neural Networks

**Clemson University** </br>
**Instructor(s):** Tim Ransom </br>

------------------------------------------------------------------------
## Learning objectives

- Define the basic components of a Convolutional Neural Network (CNN).
- Build a CNN using Keras for image classification.
- Train and test a CNN on the CIFAR-10 dataset.
- Evaluate the impact of different CNN parameters on model performance.
- Explain the advantages of using CNNs for image-related tasks.

## The goal of this lab is to introduce you to the basics of convolutional neural networks.

**Goals:**

-   Load data from the CIFAR-10 dataset using Keras
-   Build a convolutional neural network
-   Train the network
-   Test the network
-   Explore the impact of different parameters

> ⚠️ **Note:** CNNs are computationally intensive. If you are running this notebook locally, make sure you enable GPU acceleration (if available) or consider using [Google Colab](https://colab.research.google.com).

Import packages and check the version of your TensorFlow, it should be
the version \>2.0.0

In [ ]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


## What is a Convolutional Neural Network (CNN)?

A Convolutional Neural Network is a type of deep learning model particularly effective for image data. It is inspired by the way the human brain processes visual information.

### CNN Architecture Overview

A basic CNN has the following layers:

1. **Input Layer** – The image (e.g., 32x32 RGB)
2. **Convolutional Layer** – Applies filters to detect features (edges, textures, shapes)
3. **Activation Layer (ReLU)** – Introduces non-linearity
4. **Pooling Layer** – Downsamples the image (e.g., MaxPooling)
5. **Flatten Layer** – Converts 2D data to 1D
6. **Fully Connected Layer (Dense)** – Makes final predictions

> CNNs learn filters during training to detect useful patterns.


<div class="theme">Question 1:</div>

**Why are Convolutional Neural Networks (CNNs) especially suitable for image classification tasks?**

1. They apply fully connected layers to every pixel in the image.
2. CNNs are faster than traditional linear regression models for all types of data.
3. They automatically learn spatial hierarchies and local patterns in image data.
4. CNNs eliminate the need for any preprocessing of data.

**Store your answer in integer variable named `answer` in below code cell.**


In [ ]:
# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden checks here ✅🔒### 
###################################

## 1. Download, Verify and Prepare the CIFAR-10 Dataset

The CIFAR-10 dataset is a benchmark dataset used for image classification tasks. It contains **60,000 color images** across **10 mutually exclusive classes**, with 6,000 images per class.

- **Training set**: 50,000 images
- **Test set**: 10,000 images

Each image is 32x32 pixels with 3 color channels (RGB). The classes include objects like `airplane`, `cat`, `ship`, and `truck`.

Let's load the dataset using `keras.datasets`, normalize the pixel values to the [0, 1] range, and visualize a few sample images to verify the data integrity.

-----------

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0


## Visualizing CIFAR-10 Sample Images

Before we build a CNN to classify these images, it’s helpful to understand what the CIFAR-10 dataset looks like.

Each image is a **32x32 color image** (RGB) and belongs to one of the 10 mutually exclusive classes:


Let’s take a look at the first 25 images from the training set to get a sense of the data. Below each image, we’ll display the corresponding class label.


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.suptitle('Sample Images from CIFAR-10', fontsize=16)
plt.tight_layout()
plt.show()

> From this grid of images, you can notice:
> 
> - Some classes (e.g., cats, dogs) look visually similar and may be harder to distinguish.
> - Others like airplane and ship are very distinct.
> - The image resolution is low (32x32), which adds a challenge for classification models.

This preview helps us in understanding why we need convolutional filters to extract useful features from these images.


## 2. Building the Convolutional Base of a CNN

The convolutional base is the part of a CNN that performs feature extraction. It applies filters to the input image to detect patterns like edges, textures, and shapes.

We use two types of layers repeatedly:
- **Conv2D**: Applies a filter to the image
- **MaxPooling2D**: Reduces spatial dimensions (downsampling)

Each image in CIFAR-10 has shape `(32, 32, 3)`, where `3` corresponds to the RGB channels.

Let’s define a stack of convolution and pooling layers to process these images.


### Key CNN Concepts

| Component       | Description |
|----------------|-------------|
| `Conv2D`       | Applies convolution using learnable filters (kernels) |
| `filters`      | Number of output feature maps |
| `kernel_size`  | Size of the convolutional window (e.g., (3, 3)) |
| `activation`   | Usually `'relu'` in hidden layers |
| `MaxPooling2D` | Downsamples feature maps to reduce size |
| `Flatten`      | Converts 2D feature maps into 1D |
| `Dense`        | Fully connected layer for classification |


### Exercise Preview: Building a Convolution Block

Below is an example of a simple convolution block with 32 filters, followed by ReLU and max pooling:

```python
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.summary()


<div class="exercise"><b>Exercise 1: Build a CNN model with the following architecture</b> </div>

- **Input layer**: Shape `(32, 32, 3)`
- **Conv2D layer**: 32 filters, kernel size `(3,3)`, activation `'relu'`
- **MaxPooling2D**: Pool size `(2,2)`
- **Conv2D layer**: 64 filters, kernel size `(3,3)`, activation `'relu'`
- **MaxPooling2D**: Pool size `(2,2)`
- **Conv2D layer**: 64 filters, kernel size `(3,3)`, activation `'relu'`
- **Flatten layer:** to convert the tensor from `(4, 4, 64)` → `(1024,)`
- **Dense layer**: 64 units, activation `'relu'`
- **Dense layer**: 10 units (output layer for 10 CIFAR-10 classes)

Store your model in a variable named `model` and use `model.summary()` to display the architecture.


In [ ]:
"""Your code for exercise 1 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

## From the Summary of the CNN Model You Just Built

As your CNN processes images, each `Conv2D` and `MaxPooling2D` layer outputs a **3D tensor** of shape `(height, width, channels)`. These dimensions tend to **shrink in width and height** but **increase in depth (channels)** as you go deeper in the network.

For example:
- The final Conv2D output in our model is of shape `(4, 4, 64)`

### Why Flatten?
Dense layers (also called fully connected layers) expect **1D vectors** as input, not 3D tensors. Therefore, we need to **flatten** the 3D output before feeding it to Dense layers.

\[
(4, 4, 64) = 1024 \text{ elements}
\]

### Final Architecture
We add:
- A `Flatten()` layer to convert the tensor from `(4, 4, 64)` → `(1024,)`
- A hidden `Dense(64, activation='relu')` layer
- An output `Dense(10)` layer (for 10 CIFAR-10 classes)

**This final architecture allows the network to learn high-level features and make class predictions.**

### Why This Structure Works

- Stacking **Conv2D** and **MaxPooling** layers helps the model learn from simple to complex visual patterns.
- As we go deeper, we increase the **number of filters** (32 → 64 → 64) and reduce the **spatial size** of feature maps, making the model more efficient and focused.
- **Flatten + Dense** layers allow the model to **combine** all learned features and **make predictions** for each image class.

In the next step, you’ll **compile and train** this model to start making accurate predictions!

----------

<div class="theme"> Question 2:</div>
    
**Why do we use the `Flatten()` layer before the dense layers in a CNN?**

1. To remove any redundant features from the output
2. To convert 3D feature maps into 1D vectors that Dense layers can process
3. To increase the number of channels in the model
4. To apply pooling before fully connected layers

**Store your answer in integer variable named `answer` in the code cell below.**


In [ ]:
# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden checks here ✅🔒### 
###################################

## 3. Compile and Train the CNN

Now that we’ve defined our CNN, it’s time to **compile** and **train** the model.

### What Does `compile()` Do?

Before training, you need to configure our model using the `compile()` method.

- `optimizer='adam'`: Efficient optimization algorithm that adapts the learning rate
- `loss='SparseCategoricalCrossentropy'`: Suitable for multi-class classification with integer labels
- `metrics=['accuracy']`: Tracks accuracy during training and validation

### What Does `fit()` Do?

The `fit()` method starts the training process.

---------

<div class="exercise"><b>Exercise 2 : Compile your CNN model using the following configuration:</b> </div>


- **Optimizer**: `'adam'`
- **Loss Function**: `SparseCategoricalCrossentropy(from_logits=True)`
- **Metric**: `'accuracy'`

Then train the model for **10 epochs** on the CIFAR-10 training data and store the result in a variable named `model_history`.

> Note: The validation data should be `(test_images, test_labels)`.


In [ ]:
"""Your code for exercise 2.1 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

## 4. Plot and Evaluate the Model

Once our training is complete, let's evaluate our model's performance on unseen test data.

<div class="exercise"><b>Exercise 3: Using the `history` object from the previous exercise</b> </div>

1. Plot the training and validation accuracy curves.
2. Evaluate the model on the test set using `model.evaluate()`.
3. Store the test accuracy in a variable named `test_acc` and print it.

> Make sure your accuracy plot has labels and a legend.

In [ ]:
"""Your code for exercise 3 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

## Our Simple CNN Achieved About 70% Accuracy Which Is Not Bad For a Few Lines of Code !

With just a few convolutional and dense layers, our CNN has achieved over **70% accuracy** on the CIFAR-10 test set — a strong performance for a relatively small model!

This demonstrates the power of Convolutional Neural Networks (CNNs) to extract useful features from images automatically — no manual feature engineering needed!

### Why This Model Works Well

- **Convolutional layers** extract local patterns like edges and textures.
- **Pooling layers** reduce dimensionality and prevent overfitting.
- **Dense layers** at the end learn abstract combinations of features to make class predictions.

---

## How Can We Improve This Model?

To improve performance further, we can experiment with **CNN architecture and hyperparameters**, such as:

- Increasing the **number of filters** in `Conv2D` layers
- Adding more **convolutional blocks**
- Changing the **kernel size**
- Introducing **Dropout** to reduce overfitting
- Increasing the **number of epochs**
- Trying different **activation functions**
- Using **Batch Normalization**

For practice, you can experiment with changing some CNN parameters and observe their effect on test accuracy. You may download this notebook locally and try making changes to the CNN architecture or training parameters in a new code cell. Observe how these modifications impact the model’s performance.


## Conclusion: 

In this lab, you built and trained your first **Convolutional Neural Network (CNN)** for image classification using the CIFAR-10 dataset. With just a few layers and minimal configuration, your model was able to generalize well on unseen data.


---

### References:

-   [TensorFlow Tutorials](https://www.tensorflow.org/tutorials/images/cnn)
-   [TensorFlow Quick Start](https://www.tensorflow.org/tutorials/quickstart/advanced)

# END